In [0]:
import boto3
import json
import sys
import time

In [0]:
class VideoDetect:
    jobId = ''
    rek = boto3.client('rekognition', 
                       aws_access_key_id='',
    aws_secret_access_key='', 
    region_name='us-west-2')

    sqs = boto3.client('sqs', 
                       aws_access_key_id='',
    aws_secret_access_key='', 
    region_name='us-west-2')

    
    roleArn = 'arn:aws:iam::359473628279:role/Rekog'
    bucket = ''
    video = ''
    startJobId = ''
    response = ''

    sqsQueueUrl = 'https://sqs.us-west-2.amazonaws.com/359473628279/football_1'
    snsTopicArn = 'arn:aws:sns:us-west-2:359473628279:AmazonRekognition_Football'
    processType = ''

    def __init__(self, role, bucket, video):
        self.roleArn = role
        self.bucket = bucket
        self.video = video


    def GetSQSMessageSuccess(self):

        jobFound = False
        succeeded = False
    
        dotLine = 0
        while jobFound == False:
            sqsResponse = self.sqs.receive_message(QueueUrl=self.sqsQueueUrl, MessageAttributeNames=['ALL'],
                                          MaxNumberOfMessages=10)

            print(sqsResponse)
            if sqsResponse:
                
                if 'Messages' not in sqsResponse:
                    if dotLine < 40:
                        print('.', end='')
                        dotLine=dotLine+1
                    else:
                        dotLine=0    
                    sys.stdout.flush()
                    time.sleep(5)
                    continue

                for message in sqsResponse['Messages']:
                    notification = json.loads(message['Body'])
                    rekMessage = json.loads(notification['Message'])
                    print(rekMessage)
                    print(rekMessage['JobId'])
                    print(rekMessage['Status'])
                    if rekMessage['JobId'] == self.startJobId:
                        print('Matching Job Found:' + rekMessage['JobId'])
                        jobFound = True
                        if (rekMessage['Status']=='SUCCEEDED'):
                            succeeded=True

                        self.sqs.delete_message(QueueUrl=self.sqsQueueUrl,
                                       ReceiptHandle=message['ReceiptHandle'])
                    else:
                        print("Job didn't match:" +
                              str(rekMessage['JobId']) + ' : ' + self.startJobId)
                    # Delete the unknown message. Consider sending to dead letter queue
                    self.sqs.delete_message(QueueUrl=self.sqsQueueUrl,
                                   ReceiptHandle=message['ReceiptHandle'])


        return succeeded



        # ============== People pathing ===============  
    def StartPersonPathing(self):
        response=self.rek.start_person_tracking(Video={'S3Object': {'Bucket': self.bucket, 'Name': self.video}},
            NotificationChannel={'RoleArn': self.roleArn, 'SNSTopicArn': self.snsTopicArn})

        self.startJobId=response['JobId']
        print('Start Job Id: ' + self.startJobId)
    
    def GetPersonPathingResults(self):
        maxResults = 10
        paginationToken = ''
        finished = False

        while finished == False:
            response = self.rek.get_person_tracking(JobId=self.startJobId,
                                            MaxResults=maxResults,
                                            NextToken=paginationToken)

            print(response)
            print('Codec: ' + response['VideoMetadata']['Codec'])
            print('Duration: ' + str(response['VideoMetadata']['DurationMillis']))
            print('Format: ' + response['VideoMetadata']['Format'])
            print('Frame rate: ' + str(response['VideoMetadata']['FrameRate']))
            print()

            for personDetection in response['Persons']:
                print('Index: ' + str(personDetection['Person']['Index']))
                print('Timestamp: ' + str(personDetection['Timestamp']))
                print()

            if 'NextToken' in self.response:
                paginationToken = response['NextToken']
            else:
                finished = True       
       

In [0]:
def main():
   roleArn = 'arn:aws:iam::359473628279:role/Rekog'   
   bucket = 'rondale-football-videos'
   video = 'cowboys_rams_1.mov'

   analyzer=VideoDetect(roleArn, bucket,video)

   analyzer.StartPersonPathing()
   if analyzer.GetSQSMessageSuccess()==True:
        analyzer.GetPersonPathingResults()

In [105]:
main()

Start Job Id: 761973e2797271ebc24891f3f67fb76a0c38bef628e041eeb689450b4c81c0e2
{'ResponseMetadata': {'RequestId': '8f9004ab-2696-57af-ac1d-5208064e03f4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8f9004ab-2696-57af-ac1d-5208064e03f4', 'date': 'Thu, 23 Jan 2020 04:18:32 GMT', 'content-type': 'text/xml', 'content-length': '240'}, 'RetryAttempts': 0}}
.{'ResponseMetadata': {'RequestId': '80cf0516-0cb9-5b63-bead-53b52c64cf38', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '80cf0516-0cb9-5b63-bead-53b52c64cf38', 'date': 'Thu, 23 Jan 2020 04:18:37 GMT', 'content-type': 'text/xml', 'content-length': '240'}, 'RetryAttempts': 0}}
.{'ResponseMetadata': {'RequestId': 'd0c5869e-a65b-523b-8ee5-d03d86b91660', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd0c5869e-a65b-523b-8ee5-d03d86b91660', 'date': 'Thu, 23 Jan 2020 04:18:42 GMT', 'content-type': 'text/xml', 'content-length': '240'}, 'RetryAttempts': 0}}
.{'ResponseMetadata': {'RequestId': '5bc9cc68-7c4